In [1]:
!pip3 install gensim tensorflow keras pandas numpy matplotlib plotly seaborn nltk umap-learn ollama langchain langchain_community langchain_core beautifulsoup4 chromadb gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.5/612.5 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 27.4 MB/s eta 

In [7]:
import pandas as pd
import json


tweets = []
with open("/content/tweets_DM.json", "r") as file:
    for line in file:
        tweets.append(json.loads(line))


tweets_df = pd.DataFrame(tweets)


In [8]:

data_identification = pd.read_csv("/content/data_identification.csv")
emotion = pd.read_csv("/content/emotion.csv")


In [12]:

print(data_identification.columns)
print(tweets_df.columns)

print(tweets_df['_source'].head())


Index(['tweet_id', 'identification'], dtype='object')
Index(['_score', '_index', '_source', '_crawldate', '_type'], dtype='object')
0    {'tweet': {'hashtags': ['Snapchat'], 'tweet_id...
1    {'tweet': {'hashtags': ['freepress', 'TrumpLeg...
2    {'tweet': {'hashtags': ['bibleverse'], 'tweet_...
3    {'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...
4    {'tweet': {'hashtags': [], 'tweet_id': '0x2de2...
Name: _source, dtype: object


In [13]:
import pandas as pd


tweets_df = pd.json_normalize(tweets_df['_source'].apply(lambda x: x['tweet']))


In [14]:
print(tweets_df.columns)  
print(tweets_df.head())  


Index(['hashtags', 'tweet_id', 'text'], dtype='object')
                        hashtags  tweet_id  \
0                     [Snapchat]  0x376b20   
1  [freepress, TrumpLegacy, CNN]  0x2d5350   
2                   [bibleverse]  0x28b412   
3                             []  0x1cd5b0   
4                             []  0x2de201   

                                                text  
0  People who post "add me on #Snapchat" must be ...  
1  @brianklaas As we see, Trump is dangerous to #...  
2  Confident of your obedience, I write to you, k...  
3                Now ISSA is stalking Tasha 😂😂😂 <LH>  
4  "Trust is not the same as faith. A friend is s...  


In [15]:
# 合並推文內容和數據集標識
merged_data = data_identification.merge(tweets_df, on='tweet_id', how='left')
# 繼續合並情緒標簽
merged_data = merged_data.merge(emotion, on='tweet_id', how='left')
# 分離訓練和測試數據
train_df = merged_data[merged_data['identification'] == 'train']
test_df = merged_data[merged_data['identification'] == 'test']


下面開始清理推文的文本，為模型提供有效特徵。

In [16]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
   # 移除URL和用戶名等噪音
    text = re.sub(r'http\S+', '', text)  
    text = re.sub(r'@\w+', '', text)     
    text = re.sub(r'[^\w\s]', '', text)  
    text = text.lower()                  

    # 去除停用詞並進行詞形還原
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(words)

# 应用预处理
train_df['clean_text'] = train_df['text'].apply(preprocess_text)
test_df['clean_text'] = test_df['text'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-16-55248ee1ff98>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['clean_text'] = train_df['text'].apply(preprocess_text)
<ipython-input-16-55248ee1ff98>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['clean_text'] = test_df['text'].apply(preprocess_text)


使用 TF-IDF 向量化方法，将推文文本轉化为特徵向量。

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_df['clean_text'])
X_test = vectorizer.transform(test_df['clean_text'])


模型訓練

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


y_train = train_df['emotion']


model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

使用訓練好的模型對測試集進行情緒預測，生成文件。

In [20]:
# 預測情緒標簽
y_pred = model.predict(X_test)

# 創建提交文件格式
submission = test_df[['tweet_id']].copy()
submission['emotion'] = y_pred
submission.columns = ['id', 'emotion']
submission.to_csv("/content/submission.csv", index=False)
